In [30]:
import os
from dotenv import load_dotenv
from serpapi import GoogleSearch
from pydantic import Field
from typing import Optional

In [31]:
load_dotenv()
os.environ["SERPAPI_KEY"] = os.getenv("SERPAPI_KEY")

In [116]:
def get_return_flights(
    departure_token: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    ),
    arrival_id: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    ),
    departure_id: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    ),
    outbound_date: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    )
) -> list:

    params = {
        "engine": "google_flights",
        #"departure_token": departure_token,
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "currency": "GBP",
        "type": 2,
        "api_key": os.environ["SERPAPI_KEY"]
    }

    search = GoogleSearch(params)
    results = search.get_dict()["best_flights"][:3]

    info = []
    for flight in results:
        return_info = {}
        return_info["departure_info"] = flight["flights"][0]["departure_airport"]
        if "layovers" in flight:
            return_info["layovers"] = [{"name": layover["name"], "id": layover["id"]} for layover in flight["layovers"]]
        return_info["arrival_info"] = flight["flights"][-1]["arrival_airport"]
        return_info["price"] = flight["price"]
        info.append(return_info)
        
    return info

In [117]:
def get_flights(
    departure_id: str = Field(
        ..., description="The IATA code e.g. LGW for London Gatwick Airport"
    ), 
    arrival_id: str = Field(
        ..., description="The IATA code e.g. LGW for London Gatwick Airport"
    ), 
    outbound_date: str = Field(
        ..., description="The departure date in YYYY-MM-DD format"
    ), 
    return_date: Optional[str] = Field(
        None, description="The return date in YYYY-MM-DD format"
    ),
) -> dict:

        
    params = {
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "return_date": return_date,
        "currency": "GBP",
        "hl": "en",
        "api_key": os.environ["SERPAPI_KEY"]
    }

    if return_date == None:
        params["type"] = 2
    
    search = GoogleSearch(params)
    results = search.get_dict()["best_flights"][:3]

    info = {"outbound_flights":[], "return_flights":[]}
    for flight in results:
        outbound_info = {}
        outbound_info["departure_info"] = flight["flights"][0]["departure_airport"]
        if "layovers" in flight:
            outbound_info["layovers"] = [{"name": layover["name"], "id": layover["id"]} for layover in flight["layovers"]]
        outbound_info["arrival_info"] = flight["flights"][-1]["arrival_airport"]
        outbound_info["price"] = flight["price"]
        info["outbound_flights"].append(outbound_info)

    if return_date != None:
        departure_token = results[0]["departure_token"]
        return_info = get_return_flights(departure_token,departure_id,arrival_id,return_date)
        info["return_flights"].extend(return_info)


    return info

In [118]:
params = {
  "engine": "google_flights",
  "departure_id": "IST",
  "arrival_id": "PEK",
  "outbound_date": "2025-02-10",
  "return_date": "2025-02-15",
  "currency": "GBP",
  "hl": "en",
  "api_key": os.environ["SERPAPI_KEY"]
}

results = get_flights("IST","PEK","2025-02-10","2025-02-15")

In [119]:
results

{'outbound_flights': [{'departure_info': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2025-02-10 20:05'},
   'layovers': [{'name': 'Dubai International Airport', 'id': 'DXB'}],
   'arrival_info': {'name': 'Beijing Capital International Airport',
    'id': 'PEK',
    'time': '2025-02-11 14:45'},
   'price': 790},
  {'departure_info': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2025-02-10 02:10'},
   'arrival_info': {'name': 'Beijing Capital International Airport',
    'id': 'PEK',
    'time': '2025-02-10 16:15'},
   'price': 800},
  {'departure_info': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2025-02-10 16:30'},
   'layovers': [{'name': 'Suvarnabhumi Airport', 'id': 'BKK'}],
   'arrival_info': {'name': 'Beijing Capital International Airport',
    'id': 'PEK',
    'time': '2025-02-11 15:50'},
   'price': 908}],
 'return_flights': [{'departure_info': {'name': 'Beijing Capital International Airport',
    'id': 'PEK',
    'time': '2025-02-15 00: